In [1]:
import numpy as np
import pandas as pd

# --- model parameters --------------------------------------------------------
k1_true, k2_true, k4_true, k5_true = 5, 20, 1e-7, 5e-9
dt_const = 16_667          # ms  (≈ 60 Hz)

# --- synthetic‑data design ----------------------------------------------------
rng = np.random.default_rng(0)

num_Lbar          = 50    # unique L̅ levels you want in the data set
num_dL_per_Lbar   = 50     # ΔL draws you want *for each* L̅
Lbar_values       = rng.uniform(1, 100, num_Lbar)      # shape = (num_Lbar,)

# --- generate ----------------------------------------------------------------
rows = []
for Lbar in Lbar_values:
    # constants that depend only on this L̅
    a   = k1_true / (Lbar + k2_true)
    b   = k5_true * Lbar + k4_true

    # draw ΔL replicas for this L̅
    dL  = rng.normal(0, 20, num_dL_per_Lbar)
    k_dL = dL / dt_const

    μ = a * k_dL + b
    μ += rng.normal(0, 0.05 * np.abs(μ))              # 5 % noise

    σ = 0.3 / (Lbar + k2_true) * np.sqrt(Lbar) + 0.001
    σ += rng.normal(0, 0.1 * σ)                       # 10 % noise on σ

    count = rng.integers(200, 2000, num_dL_per_Lbar)

    rows.append(
        pd.DataFrame(
            {
                "MeanMin":  Lbar - 0.5,
                "MeanMax":  Lbar + 0.5,
                "DiffMin":  dL   - 0.5,
                "DiffMax":  dL   + 0.5,
                "Mu":       μ,
                "Sigma":    σ,
                "Count":    count,
            }
        )
    )

synthetic_df = pd.concat(rows, ignore_index=True)
synthetic_df.to_csv("synthetic_fit_results.csv", index=False)

print(synthetic_df.head())


     MeanMin    MeanMax    DiffMin    DiffMax            Mu     Sigma  Count
0  63.559207  64.559207   6.647608   7.647608  2.658068e-05  0.028711   1143
1  63.559207  64.559207 -24.666373 -23.666373 -8.157852e-05  0.028711    692
2  63.559207  64.559207  -0.589083   0.410917  1.015331e-07  0.028711   1910
3  63.559207  64.559207  12.629499  13.629499  4.473774e-05  0.028711   1464
4  63.559207  64.559207 -26.267229 -25.267229 -8.754300e-05  0.028711   1341
